In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_sourcedata=pd.read_csv('../sourcedata/musical_instruments_reviews_raw.csv')

In [ ]:
df_sourcedata.head()

In [ ]:
df_sourcedata.columns

In [ ]:
df_sourcedata["unixReviewTime"][:10]

In [ ]:
from datetime import datetime
df_sourcedata["unixReviewTime"].apply(lambda x: datetime.fromtimestamp(x).strftime("%Y-%m-%d-%H"))
df_sourcedata['reviewTimeInHours'] = df_sourcedata["unixReviewTime"].apply(lambda x: datetime.utcfromtimestamp(x).strftime("%Y-%m-%d-%H"))

#seems like there is no hourly data available in this dataset

In [ ]:
df_removed_dups=df_sourcedata.drop_duplicates(subset=['reviewerID', 'asin', 'reviewTimeInHours'])
print(df_sourcedata.shape)
print(df_removed_dups.shape)

In [ ]:
# assuming this means helpful[0] out of helpful[1] thinks this is helpful
#df_removed_dups['helpful']

# transform this into percentage
df_removed_dups['helpful_aslist'] = df_removed_dups['helpful'].apply(lambda x: x.strip('][').split(', '))
df_removed_dups['helpful_percentage'] = df_removed_dups['helpful_aslist'].apply(lambda x: int(x[0])/int(x[1]) if int(x[1]) > 0 else 0)


In [ ]:
#df_removed_dups.groupby('reviewTime').median()
df_mean_helpful = df_removed_dups[['reviewTimeInHours', 'helpful_percentage']].groupby('reviewTimeInHours').mean()

In [ ]:
df_median_helpful = df_removed_dups[['reviewTimeInHours', 'helpful_percentage']].groupby('reviewTimeInHours').median()

In [ ]:
#average overall rating of the product by product ID (asin), of each hourly partition
df_average_overall = df_removed_dups.groupby(['reviewTimeInHours', 'asin'])[['overall', 'helpful_percentage']].mean().reset_index()
df_average_overall.shape



In [ ]:
df_mean_overall=df_removed_dups[['asin', 'overall']].groupby('asin').mean().reset_index()
df_median_overall=df_removed_dups[['asin', 'overall']].groupby('asin').median().reset_index()
df_unique_reviews=df_removed_dups.groupby('asin')['reviewerID'].nunique().reset_index()

In [ ]:
df_removed_dups['summary_length'] = df_removed_dups['summary'].apply(lambda x: len(x))

In [ ]:
df_removed_dups[['summary', 'summary_length']]

In [ ]:
df_average_summary_length=df_removed_dups[['asin', 'summary_length']].groupby('asin').mean().reset_index()
df_median_summary_length=df_removed_dups[['asin', 'summary_length']].groupby('asin').mean().reset_index()

In [ ]:
from functools import reduce

#define list of DataFrames
dfs = [df_mean_overall, df_median_overall, df_unique_reviews, df_average_summary_length, df_median_summary_length]

#merge all DataFrames into one
output = reduce(lambda  left,right: pd.merge(left,right,on=['asin'],
                                            how='outer'), dfs)

In [ ]:
output